# Assignment 6: Custering

**THIS ASSIGNMENT IS STILL WORK-IN-PROGRESS AND SUBJECT TO CHANGE!**

[TODO: text]

## Code from previous assignments

In [1]:
import pickle, bz2
from collections import defaultdict, namedtuple, Counter
from math import log10, sqrt
from IPython.display import display, HTML
import matplotlib.pyplot as plt

# show plots inline within the notebook
%matplotlib inline
# set plots' resolution
plt.rcParams['savefig.dpi'] = 100

In [2]:
Ids_file = 'data/aspirin__Ids.pkl.bz2'
Summaries_file = 'data/aspirin__Summaries.pkl.bz2'
Abstracts_file = 'data/aspirin__Abstracts.pkl.bz2'

Ids = pickle.load( bz2.BZ2File( Ids_file, 'rb' ) )
Summaries = pickle.load( bz2.BZ2File( Summaries_file, 'rb' ) )
Abstracts = pickle.load( bz2.BZ2File( Abstracts_file, 'rb' ) )

paper = namedtuple( 'paper', ['title', 'authors', 'year', 'doi'] )

for (id, paper_info) in Summaries.items():
    Summaries[id] = paper( *paper_info )

In [3]:
def display_summary( id, show_abstract=False, show_id=True, extra_text='' ):
    """
    Function for printing a paper's summary through IPython's Rich Display System.
    Trims long author lists, and adds a link to the paper's DOI (when available).
    """
    s = Summaries[id]
    lines = []
    title = s.title
    if s.doi != '':
        title = '<a href=http://dx.doi.org/%s>%s</a>' % (s.doi, title)
    title = '<strong>' + title + '</strong>'
    lines.append(title)
    authors = ', '.join( s.authors[:20] ) + ('' if len(s.authors) <= 20 else ', ...')
    lines.append(str(s.year) + '. ' + authors)
    if (show_abstract):
        lines.append('<small><strong>Abstract:</strong> <em>%s</em></small>' % Abstracts[id])
    if (show_id):
        lines.append('[ID: %d]' % id)
    if (extra_text != ''):
         lines.append(extra_text)
    display( HTML('<br>'.join(lines)) )

TODO

In [4]:
chars = set('abcdefghijklmnopqrstuvwxy ABCDEFGHIJKLMNOPQRSTUVWXYZ')

def tokenize(text):
    return ''.join(filter(chars.__contains__, text)).split(' ')

def preprocess(tokens):
    result = []
    for token in tokens:
        result.append(token.lower())
    return result

In [5]:
inverted_index = defaultdict(set)

for (id, abstract) in Abstracts.items():
    for term in preprocess(tokenize(abstract)):
        inverted_index[term].add(id)

term_list = list(inverted_index.keys())

In [6]:
tf_matrix = defaultdict(Counter)

for (doc_id, abstract) in Abstracts.items():
    tokens = preprocess(tokenize(abstract))
    tf_matrix[doc_id] = Counter(tokens)

def tf(t,d):
    return float(tf_matrix[d][t])

def df(t):
    return float(len(inverted_index[t]))
    
num_documents = float(len(Abstracts))

def tfidf(t,d):
    return tf(t,d) * log10(num_documents/df(t))

tfidf_length_values = defaultdict(int)

for (doc_id, abstract) in Abstracts.items():
    l = 0
    for t in tf_matrix[doc_id].keys():
        l += tfidf(t,doc_id) ** 2
    tfidf_length_values[doc_id] = sqrt(l)

def tfidf_length(d):
    return tfidf_length_values[d]

In [7]:
print(len(term_list))

76653


In [8]:
short_term_list = [t for t in term_list if (len(inverted_index[t]) > 50 and len(inverted_index[t]) < 0.1*num_documents)]
print(len(short_term_list))

5020


In [9]:
import numpy as np
import lda
import lda.datasets

In [10]:
doc_term_matrix = np.zeros((len(Abstracts), len(short_term_list)), np.bool_)
doc_list = [id for (id, abstract) in Abstracts.items()]

for d in range(0,len(doc_list)):
    for i in range(0,len(short_term_list)):
        if (doc_list[d] in inverted_index[short_term_list[i]]):
            doc_term_matrix[d,i] = True

In [11]:
model = lda.LDA(n_topics=10, n_iter=500, random_state=1)
model.fit(doc_term_matrix);

INFO:lda:n_documents: 43542
INFO:lda:vocab_size: 5020
INFO:lda:n_words: 1836872
INFO:lda:n_topics: 10
INFO:lda:n_iter: 500
INFO:lda:<0> log likelihood: -19594045
INFO:lda:<10> log likelihood: -16227338
INFO:lda:<20> log likelihood: -15465971
INFO:lda:<30> log likelihood: -15154068
INFO:lda:<40> log likelihood: -15008270
INFO:lda:<50> log likelihood: -14946161
INFO:lda:<60> log likelihood: -14912860
INFO:lda:<70> log likelihood: -14893455
INFO:lda:<80> log likelihood: -14882638
INFO:lda:<90> log likelihood: -14877779
INFO:lda:<100> log likelihood: -14870883
INFO:lda:<110> log likelihood: -14868090
INFO:lda:<120> log likelihood: -14865586
INFO:lda:<130> log likelihood: -14862824
INFO:lda:<140> log likelihood: -14862720
INFO:lda:<150> log likelihood: -14861505
INFO:lda:<160> log likelihood: -14859447
INFO:lda:<170> log likelihood: -14858684
INFO:lda:<180> log likelihood: -14855308
INFO:lda:<190> log likelihood: -14854611
INFO:lda:<200> log likelihood: -14853049
INFO:lda:<210> log likeliho

In [12]:
topic_word = model.topic_word_
n_top_words = 8
for i, topic_dist in enumerate(topic_word):
    topic_words = np.array(short_term_list)[np.argsort(topic_dist)][:-n_top_words:-1]
    print('Topic {}: {}'.format(i, ' '.join(topic_words)))

Topic 0: patient case syndrome pregnancy yearold report treated
Topic 1: cells cell activity human expression cyclooxygenase role
Topic 2: should evidence agents nonsteroidal review some trials
Topic 3: cardiovascular diabetes factors age among infarction years
Topic 4: aggregation platelets inhibition thromboxane inhibited induced blood
Topic 5: gastric rats mgkg induced administration mucosal activity
Topic 6: asthma chronic response nasal function resistance respiratory
Topic 7: artery surgery myocardial infarction heparin thrombosis bypass
Topic 8: clopidogrel stroke prevention ischemic bleeding antithrombotic efficacy
Topic 9: mg p group dose groups using time


----------

# Tasks

**Your name:** ...

### Task 1

...

In [ ]:
# Add your code here

**Answer:** [_Write your answer text here_]

### Task 2

...

In [ ]:
# Add your code here

### Task 3

...

In [ ]:
# Add your code here

**Answer:** [_Write your answer text here_]

### Task 4

...

In [ ]:
# Add your code here

**Answer:** [_Write your answer text here_]

### Task 5

...

In [ ]:
# Add your code here